In [1]:
!which python

/Users/vini/Dev-Files/Poetry/virtualenvs/pyspark-ds-toolbox-H0pw_EKR-py3.8/bin/python


In [1]:
#https://github.com/manuel-calzolari/shapicant
import pandas as pd
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType, StructField, StructType, StringType
from pyspark.ml.linalg import VectorUDT

import pyspark.ml.feature as FF
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor


In [2]:
from pyspark_ds_toolbox.ml.eval import get_p1

from pyspark_ds_toolbox.ml.eval import estimate_individual_shapley_values

In [3]:
spark = SparkSession.builder\
                .appName('Ml-Pipes') \
                .master('local[1]') \
                .config('spark.executor.memory', '3G') \
                .config('spark.driver.memory', '3G') \
                .config('spark.memory.offHeap.enabled', 'true') \
                .config('spark.memory.offHeap.size', '3G') \
                .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

21/12/04 09:51:46 WARN Utils: Your hostname, matrix.local resolves to a loopback address: 127.0.0.1; using 10.0.0.105 instead (on interface en0)
21/12/04 09:51:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/04 09:51:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
def read_data(file): 
    return pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/" + file)

df = read_data('nsw_mixtape.dta')
df = pd.concat((df, read_data('cps_mixtape.dta')))
df.reset_index(level=0, inplace=True)

df = spark.createDataFrame(df.drop(columns=['data_id']))\
    .withColumn('age2', F.col('age')**2)\
    .withColumn('age3', F.col('age')**3)\
    .withColumn('educ2', F.col('educ')**2)\
    .withColumn('educ_re74', F.col('educ')*F.col('re74'))\
    .withColumn('u74', F.when(F.col('re74')==0, 1).otherwise(0))\
    .withColumn('u75', F.when(F.col('re75')==0, 1).otherwise(0))

features=['age', 'age2', 'age3', 'educ', 'educ2', 'marr', 'nodegree', 'black', 'hisp', 're74', 're75', 'u74', 'u75', 'educ_re74']
assembler = FF.VectorAssembler(inputCols=features, outputCol='features')
pipeline = Pipeline(stages = [assembler])
df_assembled = pipeline.fit(df).transform(df)

In [5]:
train_size=0.8
train, test = df_assembled.randomSplit([train_size, (1-train_size)], seed=12345)

# model = GBTRegressor(labelCol='re78')
model = GBTClassifier(labelCol='treat')
p = Pipeline(stages=[model])
p_fitted = p.fit(train)

df_predicted = p_fitted.transform(test)#.withColumn('p1', get_p1(F.col('probability')))

#df_predicted.printSchema()


# m = df_predicted.select('probability').toPandas().probability.mean()

In [6]:
df_predicted.select('index', 'features', 'prediction').show(5)

+-----+--------------------+----------+
|index|            features|prediction|
+-----+--------------------+----------+
|    3|[48.0,2304.0,1105...|       0.0|
|   12|[53.0,2809.0,1488...|       0.0|
|   12|[18.0,324.0,5832....|       0.0|
|   14|[17.0,289.0,4913....|       0.0|
|   18|(14,[0,1,2,3,4,9,...|       0.0|
+-----+--------------------+----------+
only showing top 5 rows



In [7]:
a = estimate_individual_shapley_values(
    spark=spark,
    df = df_predicted,
    id_col='index',
    model = p_fitted,
    problem_type='classification',
    row_of_interest = df_predicted.filter(F.col('index')==3).first(),
    feature_names = features,
    features_col='features',
    column_to_examine='prediction',
    print_shap_values=True
)
type(a)

Marginal Contribution for feature: age = 0.011670508861794987


Marginal Contribution for feature: age2 = -0.00015258614571412972


Marginal Contribution for feature: age3 = -0.0003122472141513387


Marginal Contribution for feature: educ = 0.010284558301219538


Marginal Contribution for feature: educ2 = -0.00011284723344770442


...

Marginal Contribution for feature: re75 = -0.004247563941649688


Marginal Contribution for feature: u74 = 0.003754619991025828


Marginal Contribution for feature: u75 = -0.00030960297110930893


Marginal Contribution for feature: educ_re74 = 0.012546462146259875


pyspark.sql.dataframe.DataFrame

In [8]:
a.select(F.sum('shap')).collect()[0][0]

-0.0165543952339246

In [11]:
df_predicted = df_predicted.withColumn('p1', get_p1(F.col('probability')))

print(df_predicted.select('p1').toPandas().p1.mean() + a.select(F.sum('shap')).collect()[0][0])

v = df_predicted.filter(F.col('index')==3).select('p1').collect()[0][0]
print(v)

0.03729094328946331
0.04363345354795456


In [20]:
df_predicted.select('re78').toPandas().re78.mean() + a.select(F.sum('shap')).collect()[0][0]

18014.104953822396

In [ ]:
print(df_predicted.select('probability').toPandas().probability.mean() + a.select(F.sum('shap')).collect()[0][0])

print(f'{v}')

0.04829068469926582
0.04364077374339104


In [ ]:
print(df_predicted.select('probability').toPandas().probability.mean() + a.select(F.sum('shap')).collect()[0][0])

print(f'{v}')

0.052278824448785434
0.043646443635225296
